# Flask App for Olympics Dashboard

#### By Safaa Amer, Maya Dennis, Christa Fields, and Kevin Yao

In [1]:
# import dependecies
from flask import Flask, jsonify, render_template
# note: importing various sqlalchemy methods for reading in sqlite database 
from sqlalchemy import create_engine, inspect, MetaData
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

In [2]:
app = Flask(__name__)



In [3]:
# sqlalchemy 'create_engine' connect to sqlite database 
engine = create_engine("sqlite:///data/newsAPI.sqlite")
#    automapping the columns to sqlalchemy base object
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [4]:
# save list of column names to var
inspector = inspect(engine)

inspector.get_table_names()

['Google_trends_results', 'Guard_api_results', 'NYT_api_results']

In [5]:
# saving references to each table
NYT_results = Base.classes.NYT_api_results
Guard_results = Base.classes.Guard_api_results
Google_results = Base.classes.Google_trends_results
print(NYT_results)

<class 'sqlalchemy.ext.automap.NYT_api_results'>


In [6]:
# creating session
session = Session(engine)

In [7]:
# Flask route: index
@app.route('/')
def index():
    return render_template("index.html")

In [8]:
# Flask route: NYT api 
@app.route('/nyt_articles')
def NYT_articles():
    NYT_ls = []
    query = session.query(NYT_results.news_id, NYT_results.year,\
                     NYT_results.article_hits).all()
    for instance in query:
        NYT_ls.append({"news_id":instance[0], "year":instance[1],\
                       "article_hits":instance[2]})
    return jsonify(NYT_ls)

In [9]:
# Flask route: theGuardian api
@app.route("/guardian_articles")
def Guard_articles():
    Guard_ls = []
    query = session.query(Guard_results.news_id, Guard_results.year,\
                     Guard_results.article_hits).all()
    for instance in query:
        Guard_ls.append({"news_id":instance[0], "year":instance[1],\
                       "article_hits":instance[2]})
    return jsonify(Guard_ls)
    

In [10]:
# Flask route: Google trends
@app.route("/google_trends")
def Google_trends():
    Goog_ls = []
    query = session.query(Google_results.month,\
                     Google_results.trend).all()
    for instance in query:
        Goog_ls.append({"month":instance[0],\
                       "trend":instance[1]})
    return jsonify(Goog_ls)
    

In [13]:
# Run app 
if __name__ == "__main__":
    app.run(debug=False)
    # if there's any problems, set debug =False

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Mar/2018 03:29:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2018 03:29:07] "GET /static/reset.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2018 03:29:07] "GET /static/index_style.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2018 03:29:07] "GET /static/news_api_and_google.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2018 03:29:07] "GET /static/900px-Olympic_flag.svg.png HTTP/1.1" 200 -
127.0.0.1 - - [16/Mar/2018 03:29:07] "GET /nyt_articles HTTP/1.1" 200 -
